In [1]:
p = 97
r = {
    'A': 14,
    'B': 38,
    'C': 92,
    'D': 69,
    'E': 70,
} 

In [2]:
polys = {
    'A': (15, 15, 2),
    'B': (95, 77, 83),
    'C': (88, 32, 18),
    'D': (62, 91, 59),
    'E': (10, 82, 52),
}   

g = {}

def generateFn(coeffs):
    def fn(x, coeffs = coeffs):
        return coeffs[0] + coeffs[1] * x + coeffs[2] * x * x
    return fn

for i, coeffs in polys.items():
    g[i] = generateFn(coeffs)

In [3]:
def computeKey(x, y):
    kxy = g[x](r[y]) % p
    kyx = g[y](r[x]) % p 
    assert(kyx == kxy)
    print("Key {0}-{1} : {2}".format(x, y, kxy))
    
from itertools import combinations

for x, y in combinations(['C', 'D'], 2):
    computeKey(x, y)
    

Key C-D : 15


In [4]:
from lib.math import *


In [9]:
import numpy as np

def decode(alliance):
    k = len(alliance)
    m = []
    b = [[] for i in range(k)]
    for l in alliance:
        row = [pow(r[l], i, p) for i in range(k)]
        m.append(row)
        for i, c in enumerate(polys[l]):
            b[i].append(c)
    ma = np.matrix(m)
    print("Matricea sistemului pentru aflarea indicilor este: \n", ma)
    print("Determinantul acesteia e")
    all_sols = []
    for i in range(k):
        sols = solveByCramer(ma, np.matrix(b[i]).transpose(), p)
        all_sols.append(sols)
    
    print("Printam coeficientii generali")
    for i, sol in enumerate(all_sols):
        for j, s in enumerate(sol):
            print("a[{0}][{1}] = {2}".format(i, j, s))
        
    def genPoly(sols):
        def poly(x, y, sols = sols):
            result = 0
            for i, sol in enumerate(sols):
                t = 0
                for j, s in enumerate(sol):
                    t += (pow(y, j, p) * s)
                    t %= p
                result += (pow(x, i, p) * t) 
                result %= p
            return result
        return poly
    
    return genPoly(all_sols)

poly = decode(['C', 'D', 'E'])
for x, y in combinations(['C', 'D'], 2):
    print("Key {0}-{1} : {2}".format(x, y, poly(r[x], r[y])))

Matricea sistemului pentru aflarea indicilor este: 
 [[ 1 92 25]
 [ 1 69  8]
 [ 1 70 50]]
Printam coeficientii generali
a[0][0] = 15
a[0][1] = 26
a[0][2] = 12
a[1][0] = 26
a[1][1] = 83
a[1][2] = 44
a[2][0] = 12
a[2][1] = 44
a[2][2] = 75
Key C-D : 15
